In [ ]:
import isomorphismUtils as iso
import json
import networkx as nx
import os
import random
import time

In [ ]:
folder_studies_path  = "RIS_fingerprint_images_folders_studies"

file_json = open(folder_studies_path +"/Center_minutiae.json",'r')
data_centers = json.load(file_json)
data_centers = data_centers['id_minutia_center']

names = list(data_centers.keys())

all_graphs = {}

for i_1 in range(0, len(names)):
    name1 = names[i_1]
    
    print(name1)

    #G1 = make_graph(folder_studies_path, name1)
    G1 = iso.make_modified_graph(folder_studies_path, name1, data_centers[name1])
    
    print(len(G1.nodes))
    print(len(G1.edges)) 

    all_graphs[name1] = G1.copy() 

In [ ]:
names_person = []

for name in names:
    if name[3:5] == 'B0' or name[3:5] == 'C0':
        print(str(name[0:5]))

    names_person.append(str(name[0:5]))

names_person = list(set(names_person))
len(names_person)

In [ ]:
names_fingers = []

for name in names:
    print(name[0:8])
    names_fingers.append(str(name[0:8]))

names_fingers = list(set(names_fingers))
len(names_fingers)

In [ ]:
count = 0
comparisons = {}
numbers_couples_comparisons = {}
count_familiar = 0
for i_1 in range(0, len(names)):
    destra_1 = False
    if int(names[i_1][6:8]) in range(0,6):
        destra_1 = True

    set_to_compare = []
    list_family = []
    for i_c in range(0, len(names)):
        destra_c = False
        if int(names[i_c][6:8]) in range(0,6):
            destra_c = True
        
        #if ((destra_1 and destra_c) or (not destra_1 and not destra_c)):
        if (names[i_1][0:3] != names[i_c][0:3]):
            if names[i_c] in comparisons:
                if names[i_1] not in comparisons[names[i_c]]:
                    set_to_compare.append(names[i_c])
            else:
                set_to_compare.append(names[i_c])
        else:
            if i_c >= i_1:
                if (names[i_1][3:5] == 'B0' and names[i_c][3:5] == 'C0') or (names[i_c][3:5] == 'B0' and names[i_1][3:5] == 'C0'):
                    if names[i_c] in comparisons:
                        if names[i_1] not in comparisons[names[i_c]]:
                            set_to_compare.append(names[i_c])
                    else:
                        set_to_compare.append(names[i_c])
                else:
                    if names[i_1] != names[i_c]:
                        list_family.append(names[i_c])
    
    if len(set_to_compare) > 11: 
        list_to_compare = random.SystemRandom().sample(set_to_compare, 11)
    
    list_to_compare = list_family + list_to_compare

    comparisons[names[i_1]] = list_to_compare

    count = count + len(list_to_compare)
    count_familiar  = count_familiar + len(list_family)

print(count - count_familiar)
print(count_familiar)
print(count)

In [ ]:
path_save_images = "RIS_fingerprint_isomorphism_random_modified_graph"

if not os.path.exists(path_save_images):
        os.makedirs(path_save_images)

count = 0
comparisons = {}
numbers_couples_comparisons = {}
count_familiar = 0
for i_1 in range(0, len(names)):
    destra_1 = False
    if int(names[i_1][6:8]) in range(0,6):
        destra_1 = True

    set_to_compare = []
    list_family = []
    for i_c in range(0, len(names)):
        destra_c = False
        if int(names[i_c][6:8]) in range(0,6):
            destra_c = True
        
        #if ((destra_1 and destra_c) or (not destra_1 and not destra_c)):
        if (names[i_1][0:3] != names[i_c][0:3]):
            if names[i_c] in comparisons:
                if names[i_1] not in comparisons[names[i_c]]:
                    set_to_compare.append(names[i_c])
            else:
                set_to_compare.append(names[i_c])
        else:
            if i_c >= i_1:
                if (names[i_1][3:5] == 'B0' and names[i_c][3:5] == 'C0') or (names[i_c][3:5] == 'B0' and names[i_1][3:5] == 'C0'):
                    if names[i_c] in comparisons:
                        if names[i_1] not in comparisons[names[i_c]]:
                            set_to_compare.append(names[i_c])
                    else:
                        set_to_compare.append(names[i_c])
                else:
                    if names[i_1] != names[i_c]:
                        list_family.append(names[i_c])
    
    if len(set_to_compare) > 11: 
        list_to_compare = random.SystemRandom().sample(set_to_compare, 11)
    
    list_to_compare = list_family + list_to_compare

    comparisons[names[i_1]] = list_to_compare

    count = count + len(list_to_compare)
    count_familiar  = count_familiar + len(list_family)

    name1 = names[i_1]
    print(name1)
    print(list_to_compare)
    
    G1 = all_graphs[name1]


    for name2 in list_to_compare:
        
        G2 = all_graphs[name2]

        print(name2)

        start_time = time.time()
        print('Inizio trovare isomorfismi')
        edges_matches, couples = iso.find_all_isomorphism(G1, G2, 3)
        print('Fine trovare isomorfismi')

        print('Trovate ' + str(len(couples)) + ' coppie di sottografi')

        numbers_couples_comparisons[(name1, name2)] = len(couples)

        subgraphs_iso_couples = couples
        name_image = 0

        path_current_images = path_save_images + '/' + name1 + '_' + name2

        if not os.path.exists(path_current_images):
            os.makedirs(path_current_images)
        
        print('Inizio stampe')
        for couple_subgraph in subgraphs_iso_couples:
            subgraphs_matching_data = {'subgraphs':
                                {'subgraph_g1':{'edges': list(couple_subgraph[0].edges), 
                                                'nodes': list(couple_subgraph[0].nodes),
                                                'edges_type': list(nx.get_edge_attributes(couple_subgraph[0], 'type').values()),
                                                'node_type': list(nx.get_node_attributes(couple_subgraph[0], 'type').values()),
                                                'node_angle': list(nx.get_node_attributes(couple_subgraph[0], 'angle').values()),
                                                'node_length': list(nx.get_node_attributes(couple_subgraph[0], 'length').values())}, 
                                    'subgraph_g2':{'edges': list(couple_subgraph[1].edges), 
                                                'nodes': list(couple_subgraph[1].nodes),
                                                'edges_type': list(nx.get_edge_attributes(couple_subgraph[1], 'type').values()),
                                                'node_type': list(nx.get_node_attributes(couple_subgraph[1], 'type').values()),
                                                'node_angle': list(nx.get_node_attributes(couple_subgraph[1], 'angle').values()),
                                                'node_length': list(nx.get_node_attributes(couple_subgraph[1], 'length').values())}},
                                    #'edges_couples': edges_matches
                                    }
            #print(subgraphs_matching_data)
            with open(path_current_images + '/'+ str(name_image) +'.json', 'w') as file_json_out:
                json.dump(subgraphs_matching_data,file_json_out)
            
            name_image = name_image + 1 
        print("--- %s seconds ---" % (time.time() - start_time))
        

#numbers_couples_comparisons

print(count - count_familiar)
print(count_familiar)
print(count)

In [ ]:
numbers_couples_comparisons

In [ ]:
same_finger = []
same_person = []
same_family = []
strangers = []
for couple in numbers_couples_comparisons:
    
    number_structures = numbers_couples_comparisons[couple]
    
    name_1 = couple[0]
    name_2 = couple[1]

    #familia
    #print(name_1[0:2])
    #persona
    #print(name_1[3:5])
    #dito
    #print(name_1[6:8])
    if name_1[0:2] == name_2[0:2]:
        if name_1[3:5] == name_2[3:5]:
            if name_1[6:8] == name_2[6:8]:
                same_finger.append(number_structures)
            else:
                same_person.append(number_structures)
        else:
            if (name_1[3:5] == 'B0' and name_2[3:5] == 'C0') or (name_1[3:5] == 'C0' and name_2[3:5] == 'B0'):
                strangers.append(number_structures)
            else:
                same_family.append(number_structures)
    else:
        strangers.append(number_structures)

print(len(same_finger))
print(len(same_person))
print(len(same_family))
print(len(strangers))

In [ ]:
print(sum(same_finger)/len(same_finger))
print(sum(same_person)/len(same_person))
print(sum(same_family)/len(same_family))
print(sum(strangers)/len(strangers))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, axs = plt.subplots(4, 1)
axs[0].hist(same_finger)
axs[0].set_title('Same finger')
axs[1].hist(same_person)
axs[1].set_title('Same person')
axs[2].hist(same_family)
axs[2].set_title('Same family')
axs[3].hist(strangers)
axs[3].set_title('Strangers')

#for ax in axs.flat:
#    ax.set(xlabel='x-label', ylabel='y-label')

fig.set_figwidth(10)
fig.set_figheight(15)

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

fig.savefig('hist_numbers_matches.png')

In [ ]:
sns.boxplot(data=same_finger)

In [ ]:
sns.boxplot(data=same_person)

In [ ]:
sns.boxplot(data=same_family)

In [ ]:
sns.boxplot(data=strangers)